This jupyter notebook aims to read the data from Google Scholar csv poll into a panda dataframe to perform basic statistics about proteomics software useage by 2019. 


In [10]:
import pandas as pd 
#import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)      

# Read the poll data 

data_df = pd.read_csv("../data/poll-results-07032019.csv")
data_df.head() 

def get_pie_chart(collection): 
    """
    This function returns a pie chart from a collection of elements. 
    """
    affiliation_counts = {} 
    for affiliation in affiliations:
        token_affiliations = str(affiliation).split(";")
        for token in token_affiliations: 
            if token.strip() != 'nan':
                if token not in affiliation_counts:
                   affiliation_counts[token] = 0
                affiliation_counts[token] = affiliation_counts[token] + 1

  
    trace=go.Pie(labels=[k  for  k in  affiliation_counts.keys()],values=[k  for  k in  affiliation_counts.values()])
    return trace

Read the position and occupation of every participant of the poll. 

In [12]:
affiliations = data_df.iloc[:,2]
trace = get_pie_chart(affiliations)
iplot([trace])

The number of participants in the poll by type of Institution. 

In [7]:
affiliations = data_df.iloc[:,3]
trace = get_pie_chart(affiliations)     
trace=go.Pie(labels=[k  for  k in  affiliation_counts.keys()],values=[k  for  k in  affiliation_counts.values()])
iplot([trace])